##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 노트북에서 TensorBoard 사용하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/tensorboard/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

TensorBoard는 [Colab](https://colab.research.google.com/) 및 [Jupyter](https://jupyter.org/)와 같은 노트북 환경에서 직접 사용할 수 있습니다. 이는 결과를 공유하고, TensorBoard를 기존 워크플로에 통합하고, 로컬에 아무것도 설치하지 않고 TensorBoard를 사용하는 데 도움이 될 수 있습니다.

## 설정

우선, TF 2.0을 설치하고 TensorBoard 노트북 확장을 로드합니다.

**Jupyter 사용자의 경우:** Jupyter와 TensorBoard를 동일한 virtualenv에 설치했다면 문제 없이 진행됩니다. 다른 Conda/virtualenv 환경에 대한 전역 Jupyter 설치 및 커널과 같이 더 복잡한 설정을 사용하는 경우에는 `tensorboard` 바이너리가 Jupyter 노트북 컨텍스트 내의 `PATH`에 있도록 해야 합니다. 이렇게 하는 한 가지 방법은 [여기의 설명에 따라](https://github.com/ipython/ipykernel/issues/395#issuecomment-479787997) `kernel_spec`을 수정하여 환경의 `bin` 디렉토리를 `PATH` 앞에 추가하는 것입니다.


**Docker 사용자의 경우**[: ](https://docs.docker.com/install/)[TensorFlow의 nightly를 사용하여 Jupyter Notebook 서버](https://www.tensorflow.org/install/docker#examples_using_cpu-only_images)의 [Docker](https://docs.docker.com/install/) 이미지를 실행하는 경우 노트북의 포트뿐만 아니라 TensorBoard의 포트도 노출해야 합니다. 따라서 다음 명령으로 컨테이너를 실행합니다.

```
docker run -it -p 8888:8888 -p 6006:6006 \
tensorflow/tensorflow:nightly-py3-jupyter
```

여기서 `-p 6006`은 TensorBoard의 기본 포트입니다. 이렇게 하면 하나의 TensorBoard 인스턴스를 실행할 수 있는 포트가 할당됩니다. 동시 인스턴스를 가지려면 더 많은 포트를 할당해야 합니다. 또한 `--bind_all`을 `%tensorboard`에 전달하여 컨테이너 외부에 포트를 노출합니다.

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

TensorFlow, datetime 및 os를 가져옵니다.

In [ ]:
import tensorflow as tf
import datetime, os

## 노트북의 TensorBoard

[FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) 데이터세트를 다운로드하고 범위를 조정합니다.

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4423680/4422102 [==============================] - 0s 0us/step


매우 간단한 모델을 만듭니다.

In [ ]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

Keras 및 TensorBoard 콜백을 사용하여 모델을 훈련합니다.

In [6]:
def train_model():
  
  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(x=x_train, 
            y=y_train, 
            epochs=5, 
            validation_data=(x_test, y_test), 
            callbacks=[tensorboard_callback])

train_model()

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 11s 182us/sample - loss: 0.4976 - accuracy: 0.8204 - val_loss: 0.4143 - val_accuracy: 0.8538
Epoch 2/5
60000/60000 [==============================] - 10s 174us/sample - loss: 0.3845 - accuracy: 0.8588 - val_loss: 0.3855 - val_accuracy: 0.8626
Epoch 3/5
60000/60000 [==============================] - 10s 175us/sample - loss: 0.3513 - accuracy: 0.8705 - val_loss: 0.3740 - val_accuracy: 0.8607
Epoch 4/5
60000/60000 [==============================] - 11s 177us/sample - loss: 0.3287 - accuracy: 0.8793 - val_loss: 0.3596 - val_accuracy: 0.8719
Epoch 5/5
60000/60000 [==============================] - 11s 178us/sample - loss: 0.3153 - accuracy: 0.8825 - val_loss: 0.3360 - val_accuracy: 0.8782


[magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html)를 사용하여 노트북 내에서 TensorBoard를 시작합니다.

In [ ]:
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard.png?raw=1"/> -->

이제 스칼라, 그래프, 히스토그램 등의 대시보드를 볼 수 있습니다. 일부 대시보드는 아직 Colab에서 사용할 수 없습니다(예: 프로필 플러그인).

`%tensorboard` 매직은 TensorBoard 명령줄 호출과 정확히 동일한 형식을 갖지만 앞에 `%` 기호가 있습니다.

훈련 전에 TensorBoard를 시작하여 진행 상황을 모니터링할 수도 있습니다.

In [ ]:
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_two_runs.png?raw=1"/> -->

동일한 TensorBoard 백엔드는 같은 명령을 실행하여 재사용됩니다. 다른 로그 디렉토리를 선택한 경우, TensorBoard의 새 인스턴스가 열립니다. 포트는 자동으로 관리됩니다.

새 모델의 훈련을 시작하고 TensorBoard가 30초마다 자동으로 업데이트되는 것을 확인하거나 오른쪽 상단의 버튼을 이용해 새로 고칩니다.

In [9]:
train_model()

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 11s 184us/sample - loss: 0.4968 - accuracy: 0.8223 - val_loss: 0.4216 - val_accuracy: 0.8481
Epoch 2/5
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3847 - accuracy: 0.8587 - val_loss: 0.4056 - val_accuracy: 0.8545
Epoch 3/5
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3495 - accuracy: 0.8727 - val_loss: 0.3600 - val_accuracy: 0.8700
Epoch 4/5
60000/60000 [==============================] - 11s 179us/sample - loss: 0.3282 - accuracy: 0.8795 - val_loss: 0.3636 - val_accuracy: 0.8694
Epoch 5/5
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3115 - accuracy: 0.8839 - val_loss: 0.3438 - val_accuracy: 0.8764


제어 수준을 좀 더 높이려면 `tensorboard.notebook` API를 사용할 수 있습니다.

In [10]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6006: logdir logs (started 0:00:54 ago; pid 265)


In [ ]:
# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000) 

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_tall.png?raw=1"/> -->